In [1]:
import sys
sys.path.append("../")
from utils import SELECTED_CODES
import numpy as np
import pandas as pd
import  cupy as cp
import numba as nb
import SharedArray as sa
from tqdm import tqdm 
from  numba import jit,prange
from mlutils.qrutils import corr

MAX_EXPR_LENGTH: 10
MAX_EPISODE_LENGTH: 256
CONSTANTS: [-5.0, -2.0, -1.0, -0.5, -0.01, 0.01, 0.5, 1.0, 2.0, 5.0]
DELTA_TIMES: [5, 8, 16, 20, 32, 40, 60, 70, 80, 90, 99, 128]
HORIZON_BACK: 200


In [2]:
def  get_data(code):
    x = sa.attach(f"X_590_{code}")
    y = sa.attach(f"label_{code}")
    ts  = sa.attach(f"timestamp_{code}")
    return  x,y,ts
with  open("/mnt/disk2/X_590/column_names.txt","r") as f:
    column_names = f.read().split("\n")

In [3]:
from concurrent.futures import ProcessPoolExecutor


def calc(code):
    x, y, z = get_data(code)
    ic = corr(x, y)
    print(f"{code} done")
    return {code: ic}


with ProcessPoolExecutor(max_workers=32) as executor:
    res = list(tqdm(executor.map(calc, SELECTED_CODES), total=len(SELECTED_CODES)))

  0%|          | 0/100 [00:00<?, ?it/s]

002058 done
300564 done
300870 done
300640 done
300867 done
300252 done
002577 done
300688 done
002308 done
300053 done
300908 done
300605 done
000925 done
002166 done
002901 done
300366 done
300713 done
000950 done
300141 done
002399 done
002498 done
300215 done
300137 done
002594 done
000537 done


  1%|          | 1/100 [03:06<5:06:54, 186.01s/it]

300498 done
002557 done
002946 done
300225 done
002941 done
000627 done


  2%|▏         | 2/100 [03:20<2:18:48, 84.99s/it] 

...

 85%|████████▌ | 85/100 [09:44<01:01,  4.11s/it]

688366 done


 95%|█████████▌| 95/100 [10:05<00:15,  3.02s/it]

689009 done
688777 done


100%|██████████| 100/100 [10:08<00:00,  6.08s/it]


In [9]:
data_dict = {}
for item in res:
    for key, value in item.items():
        data_dict[key] = value

In [10]:
df = pd.DataFrame.from_dict(data_dict, orient='index')

In [15]:
df_met = df.describe().T

In [20]:
df_met[df_met['mean']>0.01]

,count,mean,std,min,25%,50%,75%,max
2,100.0,0.022020,0.014353,-0.001579,0.011045,0.020458,0.030481,0.064603
7,100.0,0.027941,0.017392,-0.009819,0.015350,0.023938,0.037201,0.075008
8,100.0,0.019513,0.012403,-0.007736,0.010588,0.020132,0.026718,0.051017
9,100.0,0.013023,0.008955,-0.006213,0.005703,0.013589,0.018127,0.036820
10,100.0,0.010930,0.007797,-0.006807,0.004900,0.011895,0.015301,0.029874
...,...,...,...,...,...,...,...,...
570,100.0,0.021939,0.019423,-0.026095,0.007635,0.020943,0.030581,0.074847
574,100.0,0.016274,0.019845,-0.032632,0.003077,0.013958,0.028035,0.076361
582,100.0,0.010806,0.008201,-0.006139,0.003954,0.011270,0.015918,0.032437
590,100.0,0.023728,0.013952,-0.005770,0.015004,0.021769,0.031498,0.064519


In [ ]:
# # 假设我们已经有 SELECTED_CODES 和 column_names
# num_factors = len(column_names)  # 假定 column_names 已经加载

# ic_results = {}
# for code in tqdm(SELECTED_CODES):
#     x, y, ts = get_data(code)
#     daily_ic_avg = calculate_daily_ic_numba(x, y, ts, num_factors)
#     ic_results[code] = daily_ic_avg

# # 将结果转换为 Pandas DataFrame 并保存
# import pandas as pd

# results_df = pd.DataFrame.from_dict(ic_results, orient='index', columns=column_names)
